# This is a notebook for querying the Argo instance for the 1.0.0 Beacon release.
* You can run each cell individually by pressing "shift + enter".
* For more information, questions, bugs, please contact us on Slack:
    + https://join.slack.com/t/beacontechnic-wwa5548/shared_invite/zt-2dp1vv56r-tj_KFac0sAKNuAgUKPPDRg.

#### In order to get access to the Beacon endpoint, you need to fill in your unique personal token between the " " in the cell below.

In [ ]:
Token = " "

#### Install the following packages, if you have not already installed them in your environment:
* pip install requests
* pip install xarray
* pip install ipywidgets
* pip install cartopy
* pip install h5netcdf
* pip install netcdf4
* pip install scipy
* pip install packaging

#### Import the required packages

In [2]:
import requests
import json
import xarray as xr
import datetime
import pandas as pd
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

#### Retrieve the available columns from the Argo endpoint
Swagger page here: https://beacon-argo.maris.nl/swagger/

In [3]:
responseinfo = requests.get("https://beacon-argo.maris.nl/api/query/available-columns", headers = {"Authorization" : f"Bearer {Token}"}) 
params = responseinfo.json()

#### Below you can search through the available columns by entering text between the brackets of search_columns(" ").

In [4]:
def search_columns(search_term):
    search_term = search_term.lower()
    matches = [col for col in params if search_term in col.lower()]
    
    if matches:
        print("Matching columns:")
        for match in matches:
            print(match)
    else:
        print("No matching columns found.")

search_columns("temp") #Enter your search term here

Matching columns:
PROFILE_TEMP_CNDC_QC
PROFILE_TEMP_CNDC_QC._FillValue
PROFILE_TEMP_CNDC_QC.conventions
PROFILE_TEMP_CNDC_QC.long_name
PROFILE_TEMP_COUNT_DOXY_QC
PROFILE_TEMP_COUNT_DOXY_QC._FillValue
PROFILE_TEMP_COUNT_DOXY_QC.conventions
PROFILE_TEMP_COUNT_DOXY_QC.long_name
PROFILE_TEMP_CPU_CHLA_QC
PROFILE_TEMP_CPU_CHLA_QC._FillValue
PROFILE_TEMP_CPU_CHLA_QC.conventions
PROFILE_TEMP_CPU_CHLA_QC.long_name
PROFILE_TEMP_DOXY2_QC
PROFILE_TEMP_DOXY2_QC._FillValue
PROFILE_TEMP_DOXY2_QC.conventions
PROFILE_TEMP_DOXY2_QC.long_name
PROFILE_TEMP_DOXY3_QC
PROFILE_TEMP_DOXY3_QC._FillValue
PROFILE_TEMP_DOXY3_QC.conventions
PROFILE_TEMP_DOXY3_QC.long_name
PROFILE_TEMP_DOXY_MED_QC
PROFILE_TEMP_DOXY_MED_QC._FillValue
PROFILE_TEMP_DOXY_MED_QC.conventions
PROFILE_TEMP_DOXY_MED_QC.long_name
PROFILE_TEMP_DOXY_QC
PROFILE_TEMP_DOXY_QC._FillValue
PROFILE_TEMP_DOXY_QC.conventions
PROFILE_TEMP_DOXY_QC.long_name
PROFILE_TEMP_DOXY_STD_QC
PROFILE_TEMP_DOXY_STD_QC._FillValue
PROFILE_TEMP_DOXY_STD_QC.conventions
P

#### You can define here your input parameters 

In [5]:
parameter = "TEMP" #column name 
mindate = "2010-01-01" #yyyy-mm-dd
maxdate = "2011-01-01" #yyyy-mm-dd
minlon = -0
maxlon = 10
minlat = 40
maxlat = 60

#### This will create the query body based on your input parameters, you can add other "query_parameters" and "filters" to suit your needs. 
* For more query examples and explanations, you can take a look at https://maris-development.github.io/beacon/.  

In [6]:
def query(parameter, mindate, maxdate, minlon, maxlon, minlat, maxlat):
    body = {
        "query_parameters": [
            {
                "column_name": parameter,
                "alias": parameter,
                "skip_fill_values": True
            },
            {
                "column_name": f"{parameter}_QC",
                "alias": f"{parameter}_QC"
            },
            {
                "column_name": f"{parameter}.units",
                "alias": "Unit"
            },
            {
                "column_name": "cf_datetime",
                "alias": "datetime"
            },
            {
                "column_name": "PRES",
                "alias": "PRES"
            },
            {
                "column_name": "PRES_QC",
                "alias": "PRES_QC"
            },
            {
                "column_name": "LONGITUDE",
                "alias": "LONGITUDE"
            },
            {
                "column_name": "LATITUDE",
                "alias": "LATITUDE"
            },
            {
                "column_name": "WMO_INST_TYPE",
                "alias": "WMO_INST_TYPE"
            },
            {
                "column_name": "PLATFORM_NUMBER",
                "alias": "PLATFORM_NUMBER"
            },
            {
                "column_name": "CYCLE_NUMBER",
                "alias": "CYCLE_NUMBER"
            }
        ],
        "filters": [
            {
                "for_query_parameter": "datetime",
                "min": f"{mindate}T00:00:00",
                "max": f"{maxdate}T00:00:00",
                "cast": "timestamp"
            },
            {
              "for_query_parameter": "LONGITUDE",
              "min": minlon,
              "max": maxlon
            },
            {
              "for_query_parameter": "LATITUDE",
              "min": minlat,
              "max": maxlat
            }
        ],
        "output": {
            "format": {
                "odv": {
                    "longitude_column": "LONGITUDE",
                    "latitude_column": "LATITUDE",
                    "timestamp_column": {
                        "data_column_name": "datetime",
                        "comment": ""
                    },
                    "depth_column": {
                        "data_column_name": "PRES",
                        "comment": "",
                        "qf_column": "PRES_QC"
                    },
                    "data_columns": [
                        {
                            "data_column_name": parameter,
                            "comment": "",
                            "qf_column": f"{parameter}_QC"
                        }
                    ],
                    "metadata_columns": [
                        "WMO_INST_TYPE", "PLATFORM_NUMBER", "CYCLE_NUMBER"
                    ],
                    "skip_odv_script": False,
                    "qf_schema": "ARGO"
                }
            }
        }}
    return body

query_body = query(parameter, mindate, maxdate, minlon, maxlon, minlat, maxlat)

#### This is the post request that is sent to Beacon with the above specified body.

In [7]:
response = requests.post("https://beacon-argo.maris.nl/api/query", json.dumps(query_body), headers = {
    "Authorization" : f"Bearer {Token}",
    "Content-type": "application/json"
})

if response.status_code == 204:
    print("No data has been found for your query, please update your input fields above and run the notebook again.")
elif response.status_code != 200:
    print(response.text)

In [8]:
regionname = f"[{minlat},{minlon}],[{maxlat},{maxlon}]" 

if not os.path.exists("./Beacon_V1.0.0_Output"):
    os.makedirs("Beacon_V1.0.0_Output")
    
open(f'./Beacon_V1.0.0_Output/ARGO_{parameter}_{regionname}_{mindate}-{maxdate}.zip', 'wb').write(response.content);